# Sentinel-2 Collection 1, L2A <img align="right" src="../../resources/csiro_easi_logo.png">

#### Index
- [Overview](#Overview)
- [Setup (imports, defaults, dask, odc)](#Setup)
- [Example query](#Example-query)
- [Product definition](#Product-definition)
- [Quality layer](#Quality-layer)
- [Create and apply a good quality pixel mask](#Create-and-apply-a-good-quality-pixel-mask)
- [Plot and browse the data](#Plot-and-browse-the-data)

## Overview

Sentinel-2 is an Earth observation mission from the EU Copernicus Programme that systematically acquires optical imagery at high spatial resolution (up to 10 m for some bands). The mission is a constellation of two identical satellites in the same orbit, 180° apart for optimal coverage and data delivery. Together, they cover all Earth's land surfaces, large islands, inland and coastal waters every 3-5 days.

Sentinel-2A was launched on 23 June 2015 and Sentinel-2B followed on 7 March 2017.
Both of the Sentinel-2 satellites carry a wide swath high-resolution multispectral imager with 13 spectral bands.
For more information see:
- [ESA Sentinel missions](https://www.esa.int/Applications/Observing_the_Earth/Copernicus/The_Sentinel_missions)
- [Sentinel-2 technical specifications](https://sentinels.copernicus.eu/web/sentinel/technical-guides/sentinel-2-msi)

_Selected text adapted from https://github.com/GeoscienceAustralia/dea-notebooks/blob/develop/DEA_datasets/Sentinel_2.ipynb_

#### Data source and documentation

ESA produces a surface reflectance "S2 Collection 1 L2A" product using their [sen2cor](https://step.esa.int/main/snap-supported-plugins/sen2cor/) software. [Element84](https://github.com/Element84/earth-search) convert these data to Cloud-Optimized Geotiff format and makes them publicly available at their [Earth Search STAC API](https://earth-search.aws.element84.com/v1) endpoint for programmatic access.

EASI uses its STAC indexing tools to index datasets into our ODC databases.

| Name | Product | Source | Information | Index
|--|--|--|--|--|
| Sentinel-2 C1 L2A COGs | `sentinel_2_c1_l2a` | [Earth Search STAC](https://earth-search.aws.element84.com/v1/collections/sentinel-2-c1-l2a) | Use for global (land) surface reflectance | Select COGS via STAC and convert to ODC metadata

#### Collection 1 baseline processing

ESA is reprocessing Sentinel-2 to a ["Collection 1"](https://sentinels.copernicus.eu/web/sentinel/sentinel-data-access/sentinel-products/sentinel-2-data-products/collection-1-level-2a) product using processing baseline >=5.00.

Element-84 are kindly processing these data to COGs. Their processing documentation and advice is expected to be made available once the full repreocessing is completed by ESA and Element84 (see [issues](https://github.com/Element84/earth-search/issues)).

In the meantime this notebook will show how we think the data should be loaded and used.

## Setup

#### Imports

In [ ]:
# Basic plots
%matplotlib inline
# import matplotlib.pyplot as plt
# plt.rcParams['figure.figsize'] = [12, 8]

# Common imports and settings
import os, sys
os.environ['USE_PYGEOS'] = '0'
from IPython.display import Markdown
import pandas as pd
pd.set_option("display.max_rows", None)
import xarray as xr

# Datacube
import datacube
from datacube.utils.aws import configure_s3_access
import odc.geo.xr                                  # https://github.com/opendatacube/odc-geo
from datacube.utils import masking  # https://github.com/opendatacube/datacube-core/blob/develop/datacube/utils/masking.py
from odc.algo import enum_to_bool                  # https://github.com/opendatacube/odc-tools/blob/develop/libs/algo/odc/algo/_masking.py
from dea_tools.plotting import display_map, rgb    # https://github.com/GeoscienceAustralia/dea-notebooks/tree/develop/Tools

# Holoviews, Datashader and Bokeh
import hvplot.pandas
import hvplot.xarray
import panel as pn
import colorcet as cc
import cartopy.crs as ccrs
from datashader import reductions
from holoviews import opts
# hv.extension('bokeh', logo=False)

In [ ]:
# EASI defaults
# These are convenience functions so that the notebooks in this repository work in all EASI deployments
# The `git.Repo()` part returns the local directory that easi-notebooks has been cloned into
# If using the `easi-tools` functions from another path, replace `repo` with your local path to `easi-notebooks` directory

import git
repo = git.Repo('.', search_parent_directories=True).working_tree_dir   # Path to this cloned local directory
if repo not in sys.path: sys.path.append(repo)                          # Add the local path to `easi-notebooks` to python
from easi_tools import EasiDefaults
from easi_tools import initialize_dask, xarray_object_size, mostcommon_crs, heading

#### EASI defaults

These default values are configured for each EASI instance. They help us to use the same training notebooks in each EASI instance. You may find some of the functions convenient for your work or you can easily override the values in your copy of this notebook.

In [ ]:
easi = EasiDefaults()

family = 'sentinel-2'
product = 'sentinel_2_c1_l2a'
display(Markdown(f'Default {family} product for "{easi.name}": [{product}]({easi.explorer}/products/{product})'))

#### Dask cluster

Its nearly always worth starting a dask cluster as it can improve data load and processing speed.

In [ ]:
# Local cluster
cluster, client = initialize_dask(workers=4)
display(client)

# Or use Dask Gateway - this may take a few minutes
# cluster, client = initialize_dask(use_gateway=True, workers=4)
# display(client)

#### ODC database

Connect to the ODC database. Configure the environment and low-level tools to read from AWS buckets.

In [ ]:
dc = datacube.Datacube()

# Access AWS "requester-pays" buckets
# This is necessary for reading data from most third-party AWS S3 buckets such as for Landsat and Sentinel-2
configure_s3_access(aws_unsigned=False, requester_pays=True, client=client);

## Example query

Change any of the parameters in the `query` object below to adjust the location, time, projection, or spatial resolution of the returned datasets.

Use the Explorer interface to check the temporal and spatial coverage for each product.

In [ ]:
# Explorer link
display(Markdown(f'See: {easi.explorer}/products/{product}'))

# EASI defaults
display(Markdown(f'#### Location: {easi.location}'))
latitude_range = easi.latitude
longitude_range = easi.longitude
time_range = easi.time

# Or set your own latitude / longitude
# latitude_range = (21.5, 23.5)
# longitude_range = (88, 90.8)
# time_range = ('2022-01-01', '2022-03-01')

query = {
    'product': product,       # Product name
    'x': longitude_range,     # "x" axis bounds
    'y': latitude_range,      # "y" axis bounds
    'time': time_range,       # Any parsable date strings
}

# Convenience function to display the selected area of interest
display_map(longitude_range, latitude_range)

#### Most common CRS

Sentinel-2 datasets are stored with different coordinate reference systems (CRS), corresponding to the multiple UTM zones that are used for S2 L1B tiling. S2 measurement bands also have different resolutions (10 m, 20 m and 60 m). As such S2 queries need to include the following two query parameters:

* `output_crs` - This sets a consistent CRS that all Sentinel-2 data will be reprojected to, irrespective of the UTM zone the individual image is stored in.
* `resolution` - This sets the resolution that all Sentinel-2 images will be resampled to. 

Use `mostcommon_crs()` to select a CRS. Adapted from https://github.com/GeoscienceAustralia/dea-notebooks/blob/develop/Tools/dea_tools/datahandling.py

In [ ]:
# Most common CRS
native_crs = mostcommon_crs(dc, query)
print(f'Most common native CRS: {native_crs}')

# Target xarray parameters
# - Select a set of measurements to load
# - output CRS and resolution
# - Usually we group input scenes on the same day to a single time layer (groupby)
# - Select a reasonable Dask chunk size (this should be adjusted depending on the
#   spatial and resolution parameters you choose
load_params = {
    'measurements': ['blue', 'red', 'green', 'nir', 'scl'],  # Selected measurement or alias names
    'output_crs': native_crs,                       # Target EPSG code
    'resolution': (-20, 20),                        # Target resolution
    'group_by': 'solar_day',                        # Scene grouping
    'dask_chunks': {'x': 2048, 'y': 2048},          # Dask chunks
}

In [ ]:
# Load data

data = dc.load(**(query | load_params))
display(xarray_object_size(data))
display(data)

In [ ]:
# When happy with the shape and size of chunks, persist() the result
data = data.persist()

In [ ]:
# Optional

# Create a simple plot to verify that the data look reasonable
# This will load and create images from the data, which may take a few minutes
# Here we limit this plot to the first few time layers.

data.isel(time=slice(0,4)).red.plot.imshow(col="time")

## Product definition

The product definition contains details on the measurements and quality layers available in the product. Datacube provides convenience functions that return this information in `pandas DataFrames`.

Use `list_measurements` to show the details for a product, and `masking.describe_variable_flags` to show the flag definitions.

In [ ]:
# Measurement definitions for the selected product
measurement_info = dc.list_measurements().loc[query['product']]
heading(f'Measurement table for product: {query["product"]}')
display(measurement_info)

# Flag definitions
flag_name = 'scl'
heading(f'Flag definition table for flag name: {flag_name}')
display(masking.describe_variable_flags(data[flag_name]))

flags_def = masking.describe_variable_flags(data[flag_name]).loc['qa']['values']
display(flags_def)

#### Apply the correct _offset_ to the source data

ESA introduced a change to their [L1C processing](#Collection-1-baseline-processing) that encodes their L1C and L2A products with _scale_ and _offset_ value such that
`phyiscal_value = encoded_value * scale_factor + offset`. The scale and offset details per band are available in the product definition.

In [ ]:
# Apply scale and offset

for vv in data.data_vars:
    scale = measurement_info.loc[vv, 'scale_factor']
    offset = measurement_info.loc[vv, 'add_offset']
    if not pd.isnull(scale) and not pd.isnull(offset):
        data[vv] = data[vv] * scale + offset

## Quality layer

To visualise the **SCL** layer we use `hvplot` to create a dynamic (zoom, scroll) image with a colour map that corresponds to that used by ESA.

In [ ]:
# Make SCL image
# https://sentinel.esa.int/web/sentinel/technical-guides/sentinel-2-msi/level-2a/algorithm
# https://www.sentinel-hub.com/faq/how-get-s2a-scene-classification-sentinel-2/

from bokeh.models.tickers import FixedTicker

color_def = [
    (0,  '#000000', 'No data'),   # black
    (1,  '#ff0004', 'Saturated or defective'),   # red
    (2,  '#868686', 'Topographic and casted shadow'),   # gray
    (3,  '#774c0b', 'Cloud shadows'),   # brown
    (4,  '#10d32d', 'Vegetation'),   # green
    (5,  '#ffff53', 'Not vegetated'),   # yellow
    (6,  '#0000ff', 'Water'),   # blue
    (7,  '#818181', 'Unclassified'),   # medium gray
    (8,  '#c0c0c0', 'Cloud medium probability'),   # light gray
    (9,  '#f2f2f2', 'Cloud high probability'),   # very light gray
    (10, '#53fff9', 'Thin cirrus'),   # light blue/purple
    (11, '#ff52ff', 'Snow or ice'),   # cyan
]
color_val = [x[0] for x in color_def]
color_hex = [x[1] for x in color_def]
color_txt = [f'{x[0]:2d}: {x[2]}' for x in color_def]
color_lim = (min(color_val), max(color_val) + 1)
bin_edges = color_val + [max(color_val) + 1]
bin_range = (color_val[0] + 0.5, color_val[-1] + 0.5)  # No idea why (0.5,11.5) works and (0,11) or (0,12) do not

# These options manipulate the color map and colorbar to show the categories for this product
options = {
    'title': f'Flag data for: {query["product"]} ({flag_name})',
    'cmap': color_hex,
    'clim': color_lim,
    'color_levels': bin_edges,
    'colorbar': True,
    'width': 800,
    'height': 450,
    'aspect': 'equal',
    'tools': ['hover'],
    'colorbar_opts': {
        'major_label_overrides': dict(zip(color_val, color_txt)),
        'major_label_text_align': 'left',
        'ticker': FixedTicker(ticks=color_val),
    },
}

# Set the dataset CRS, if using hvplot's projection and coastlines options
# plot_crs = native_crs
# if plot_crs == 'epsg:4326':
#     plot_crs = ccrs.PlateCarree()

# Native data and coastline overlay:
# - Comment `crs`, `projection`, `coastline` to plot in native_crs coords
# TODO: Update the axis labels to 'longitude', 'latitude' if `coastline` is used

quality_plot = data[flag_name].hvplot.image(
    x = 'x', y = 'y',                        # Dataset x,y dimension names
    rasterize = True,                        # Use Datashader
    aggregator = reductions.mode(),          # Datashader selects mode value, requires 'hv.Image'
    precompute = True,                       # Datashader precomputes what it can
    # crs = plot_crs,                          # Datset crs
    # projection = ccrs.PlateCarree(),         # Output projection (ccrs.PlateCarree() when coastline=True)
    # coastline = '10m',                       # Coastline = '10m'/'50m'/'110m'
).options(opts.Image(**options)).hist(bin_range = bin_range)

# display(quality_plot)
# Optional: Change the default time slider to a dropdown list, https://stackoverflow.com/a/54912917
fig = pn.panel(quality_plot, widgets={'time': pn.widgets.Select})  # widget_location='top_left'
fig

## Create and apply a good quality pixel mask

Select a set of flag values that represent "good quality" for your application. Here we select "vegetation", "not vegetated" and "water"; that is we exclude clouds and low-quality features.

The **SCL** layer uses distinct integer values to represent each class. The datacube `enum_to_bool()` function creates a boolean mask layer corresponding to a set of category values (string names).

Recall that *scale* and *offset* (if required) have already been applied by the `load_s2l2a_with_offset()` function.

In [ ]:
# Create Mask layer

good_pixel_flags = [flags_def[str(i)] for i in [4, 5, 6]]

good_pixel_mask = enum_to_bool(data[flag_name], good_pixel_flags)
display(good_pixel_mask)  # -> DataArray. Type: bool

# Apply good pixel mask (multiple layers)
good_data = data.where(good_pixel_mask).persist()

## Plot and browse the data

There are numerous tools we can use to plot and interact with the data. Here we use `hvplot` again because it works well with dask and allows us to zoom and scroll quite efficiently. `Hvplot` uses [Datashader](https://datashader.org/getting_started/Pipeline.html) to process and render only the pixels that are required for the viewport.

Various options can be changed such as the data layer, colour map and colour range.

In [ ]:
# Create an NDVI layer, as useful way to visualise the data for differences in vegetation and land cover
# ndvi = (nir - red) / (nir + red)

ndvi = (good_data.nir - good_data.red) / (good_data.nir + good_data.red)
ndvi.persist()

In [ ]:
# Generate a plot

options = {
    'title': f'{query["product"]}',
    'width': 800,
    'height': 450,
    'aspect': 'equal',
    'cmap': cc.rainbow,
    'clim': (0, 1),                          # Limit the color range depending on the layer_name
    'colorbar': True,
    'tools': ['hover'],
}

# Set the dataset CRS, if using hvplot's projection and coastlines options
# plot_crs = native_crs
# if plot_crs == 'epsg:4326':
#     plot_crs = ccrs.PlateCarree()

# Native data and coastline overlay:
# - Comment `crs`, `projection`, `coastline` to plot in native_crs coords
# TODO: Update the axis labels to 'longitude', 'latitude' if `coastline` is used

layer_plot = ndvi.hvplot.image(
    x = 'x', y = 'y',                        # Dataset x,y dimension names
    rasterize = True,                        # Use Datashader
    aggregator = reductions.mean(),          # Datashader selects mean value
    precompute = True,                       # Datashader precomputes what it can
    # crs = plot_crs,                        # Dataset crs
    # projection = ccrs.PlateCarree(),         # Output projection (use ccrs.PlateCarree() when coastline=True)
    # coastline='10m',                         # Coastline = '10m'/'50m'/'110m'
).options(opts.Image(**options)).hist(bin_range = options['clim'])

# display(layer_plot)
# Optional: Change the default time slider to a dropdown list, https://stackoverflow.com/a/54912917
fig = pn.panel(layer_plot, widgets={'time': pn.widgets.Select})  # widget_location='top_left'
display(fig)